# Multiword expressions identification and extraction

The task shows two simple methods useful for identifying multiword expressions (MWE) in corpora.

## Tasks

1. Use SpaCy [tokenizer API](https://spacy.io/api/tokenizer) to tokenize the text from the law corpus.

In [1]:
import os
import pickle

# python -m spacy download en_core_web_sm
# python -m spacy download pl_core_news_sm
import re
import string
import tarfile
from collections import Counter

import matplotlib
import matplotlib.pyplot as plt
import morfeusz2
import numpy as np
import pandas as pd
import regex
import spacy
from elasticsearch import *
from elasticsearch.helpers import *
from elasticsearch_dsl import *
from elasticsearch_dsl import query
from spacy.tokenizer import *

matplotlib.style.use("ggplot")
import time

import Levenshtein

%matplotlib inline
import pandas as pd


2. Compute **bigram** counts of downcased tokens.  Given the sentence: "The quick brown fox jumps over the
   lazy dog.", the bigram counts are as follows:
   
   * "the quick": 1
   * "quick brown": 1
   * "brown fox": 1
   * . ...
   * "dog .": 1

   
3. Discard bigrams containing characters other than letters. Make sure that you discard the invalid entries **after**
   computing the bigram counts.

4. Use [pointwise mutual information](https://en.wikipedia.org/wiki/Pointwise_mutual_information) to compute the measure 
   for all pairs of words. 

5. Sort the word pairs according to that measure in the descending order and determine top 10 entries.

6. Filter bigrams with number of occurrences lower than 5. Determine top 10 entries for the remaining dataset (>=5
   occurrences).

7. Use [log likelihood ratio](http://tdunning.blogspot.com/2008/03/surprise-and-coincidence.html) (LLR) to compute the measure
   for all pairs of words.

8. Sort the word pairs according to that measure in the descending order and display top 10 entries.

9. Compute **trigram** counts for the whole corpus and perform the same filtering.

10. Use PMI (with 5 occurrence threshold) and LLR to compute top 10 results for the trigrams. Devise a method for computing the values, based on the
   results for bigrams.

11. Create a table comparing the methods (separate table for bigrams and trigrams).

12. Answer the following questions:

   a. Why do we have to filter the bigrams, rather than the token sequence?
   
   b. Which measure (PMI, PMI with filtering, LLR) works better for the bigrams and which for the trigrams?
   
   c. What types of expressions are discovered by the methods.
   
   d. Can you devise a different type of filtering that would yield better results?